
# **Setup Environment Script Guide**

* This script sets up the execution environment for running code both **locally** (Jupyter, terminal) and **remotely** (Google Colab). It ensures that the necessary directories are accessible and allows optional GPU usage when running locally.
* Then, the script shows **how to call this method in different ways**

## **How It Works**
- The function `setup_environment(curr_proj_folder, use_gpu=True)`:
  - Determines whether the script is running locally or in **Google Colab**.
  - If running on **Colab**, it **mounts Google Drive**, sets the working directory, and returns `mount_success` to indicate if mounting was successful.
  - If running **locally**, it verifies the folder path and checks for GPU availability.

## **Return Values**
- **run_local (bool)**: `True` if running locally, `False` if running on Colab.
- **path_to_scripts (str)**: The path to the working directory.
- **mount_success (bool)**: `True` if Google Drive was successfully mounted (only relevant for Colab).

## **Usage Scenarios**
### **1. Running Locally (Jupyter, Terminal)**
```python
run_local, script_path, mount_success = setup_environment(curr_proj_folder="my_project", use_gpu=True)
```
- Ensures the current directory matches the project folder.
- Checks for **GPU availability** (via `nvidia-smi`).
- If a GPU is available and `use_gpu=True`, it will be used.

### **2. Running on Google Colab**
```python
run_local, script_path, mount_success = setup_environment(curr_proj_folder="my_project", use_gpu=False)
```
- **Mounts Google Drive** at `/content/drive`.
- Navigates to the project folder (`Colab Notebooks/my_project`).
- `mount_success` is `True` if Drive was successfully mounted.

## **Handling GPU Selection in Colab**
Currently, `use_gpu` is only relevant when running locally. In **Google Colab**, you must manually select a GPU:
1. **Go to** `Runtime` > `Change runtime type`.
2. **Set Hardware Accelerator** to `GPU`.

## **Unmounting Drive in Colab**
At the end of the session, you can unmount the drive **only if it was successfully mounted**:
```python
from google.colab import drive
if not run_local and mount_success:
    drive.flush_and_unmount()
    print("Drive unmounted successfully!")
```

## **Conclusion**
This script provides a seamless way to manage execution environments for both **local and Colab runs**, ensuring proper directory setup and GPU availability.


In [ ]:
import os
import subprocess
from typing import Tuple

def setup_environment(curr_proj_folder: str = "pretraining-llms", use_gpu: bool = True) -> Tuple[bool, str]:
    """
    Sets up the environment for running code, handling local and remote execution.

    Args:
        curr_proj_folder (str, optional): Folder name of the current project. Defaults to "pretraining-llms".
        use_gpu (bool, optional): Whether to use GPU if available. Defaults to True.

    Returns:
        Tuple[bool, str,bool]: (run_local status, computed path_to_scripts,mount_success status)
    """
    try:
      from google.colab import drive
      run_local = False
    except ImportError:
      run_local = True

    google_drive_base_folder = 'Colab Notebooks'
    print("Running Remote") if not run_local else print("Running locally")

    # Initialize mount status for Colab
    mount_success = False
    # Remote run code
    if not run_local:
      # Mount Google Drive
      drive.mount('/content/drive')
      # Check if the mount was successful
      if os.path.ismount('/content/drive'):
        print("Google Drive mounted successfully!")
        mount_success = True
      else:
        print("Drive mount failed.")
      # By Default, this is complete mount path
      mount_path = '/content/drive/MyDrive'

      if curr_proj_folder is None:
        curr_proj_folder = 'pretraining-llms'
      # complete path to current files
      path_to_scripts = os.path.join(mount_path, google_drive_base_folder,curr_proj_folder)
      # Create the directory if it doesn't exist
      if not os.path.exists(path_to_scripts):
        os.makedirs(path_to_scripts)
        # change to the path
      os.chdir(path_to_scripts)
      print(f'Running code in path {os.getcwd()}')
    # Local Run
    else:
      path_to_scripts  = os.getcwd()
      if curr_proj_folder:
          # folder name provided as argument should match the one existing
          assert os.path.basename(path_to_scripts ) == curr_proj_folder, \
          f"Folder Name Mismatch: {os.path.basename(path_to_scripts )} != {curr_proj_folder}"
      # in this case, curr_proj_folder is not required, as its computed automatically

      print(f'Running code in path {path_to_scripts }')
      if use_gpu:
        try:
          gpu_info = subprocess.check_output("nvidia-smi", shell=True).decode('utf-8')
          print("******GPU is available and will be used:**********")
          print(gpu_info)
        except subprocess.SubprocessError:
          print("GPU check failed (nvidia-smi not found or no GPU available). Falling back to CPU.")
          use_gpu = False  # Force CPU usage if GPU check fails
      else:
          print("******use_gpu is set to False. Using CPU******")
    return run_local, path_to_scripts,mount_success



In [ ]:
# Example usage
run_local, script_path,mount_success = setup_environment(curr_proj_folder = "pretraining-llms",use_gpu = True)
#run_local, script_path,_ = setup_environment(use_gpu = True)
#run_local, script_path,_ = setup_environment()
#run_local, script_path,_ = setup_environment(curr_proj_folder = "pretrain-llms",use_gpu = True)
print(f"Run Local: {run_local}, Script Path: {script_path}")

In [ ]:
 if not run_local and mount_success:
    from google.colab import drive
    drive.flush_and_unmount()
    print("Unmounted Google Drive.")

In [ ]:
run_local, script_path,_ = setup_environment(curr_proj_folder = "pretraining-llms",use_gpu = True)